In [76]:
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from PIL import Image
# label_index = {'landfill_trash': 6, 'plastic': 5, 'paper': 4, 'mental': 3, 'glass': 2, 'cardboard': 1};


In [77]:
model_vgg = VGG16(weights='imagenet', include_top=False)

In [78]:
def modelProcess(img_path, model):
    img = load_img(img_path, target_size=(224, 224)) 
    img = img_to_array(img) 
    x = np.expand_dims(img, axis=0) 
    x = preprocess_input(x) 
    x_vgg = model.predict(x) 
    
    x_vgg = x_vgg.reshape(1, 25088)
    return x_vgg

In [79]:
def transform_format(path):  
    folders = os.listdir(path)  
    for j in range(len(folders)):
        dirName = path + '//' + folders[j] + '//' 
        li = os.listdir(dirName)  
        for filename in li:
            newname = filename
            newname = newname.split(".")  
            if newname[-1] != "png":  
                newname[-1] = "png"
                newname = str.join(".", newname)  
                filename = dirName + filename
                newname = dirName + newname
                os.rename(filename, newname)  
                print('reading the images:%s' % (newname))  
                a = np.array(Image.open(newname))  
                if ((len(a.shape) != 3) or (a.shape[2] != 3)):  
                    a = np.array(Image.open(newname).convert('RGB'))  
                    img = Image.fromarray(a.astype('uint8'))  
                    img.save(newname)  
                    print(a.shape)  

In [80]:
def read_data(path):
    folders = os.listdir(path) 
    for j in range(len(folders)): 
        folder = path + '//' + folders[j] 
        dirs = os.listdir(folder) 
       
        img_path = []
        for i in dirs:
            if os.path.splitext(i)[1] == ".png": 
                img_path.append(i)
        img_path = [folder + "//" + i for i in img_path] 
        features1 = np.zeros([len(img_path), 25088]) 
        for i in range(len(img_path)):
            feature_i = modelProcess(img_path[i], model_vgg) 
            # print('preprocessed:', img_path[i])
            features1[i] = feature_i
        if j == 0: 
            X = features1 
        else:
            X = np.concatenate((X, features1), axis=0)
           
    return X 
       

In [81]:
def read_label(path):
    y = []
    folders = os.listdir(path)  
    for j in range(len(folders)): 
        dirName = path + '//' + folders[j] + '//' 
        lens = len(os.listdir(dirName)) 
        for i in range(lens):
            y.append(j) 
    lb = LabelBinarizer() 
    y = lb.fit_transform(y)
    return y

In [82]:
path = '/Users/sheny/Desktop/Module3/TR_NEW' 

y = []
folders = os.listdir(path) 
for j in range(len(folders)): 
    dirName = path + '//' + folders[j] + '//'  
    lens = len(os.listdir(dirName)) 
    for i in range(lens):
        y.append(j) 

lb = LabelBinarizer()
y = lb.fit_transform(y) 

transform_format(path)  
X = read_data(path) 


In [83]:
print(folders)

['cardboard', 'glass', 'landfill_trash', 'mental', 'paper', 'plastic']


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (1010, 25088)
X_test.shape: (253, 25088)
y_train.shape: (1010, 6)
y_test.shape: (253, 6)


In [87]:
# from keras.layers.core import Dropout

In [88]:
model = Sequential()
model.add(Dense(units=40, activation='relu', input_dim=25088))

model.add(Dense(units=6, activation='softmax'))

model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 40)                1003560   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 246       
_________________________________________________________________
dropout_2 (Dropout)          (None, 6)                 0         
Total params: 1,003,806
Trainable params: 1,003,806
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=39)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/39
1010/1010 [==============================] - 0s 345us/step - loss: 0.7604 - accuracy: 0.1693
Epoch 2/39
1010/1010 [==============================] - 0s 202us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 3/39
1010/1010 [==============================] - 0s 199us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 4/39
1010/1010 [==============================] - 0s 198us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 5/39
1010/1010 [==============================] - 0s 198us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 6/39
1010/1010 [==============================] - 0s 197us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 7/39
1010/1010 [==============================] - 0s 202us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 8/39
1010/1010 [==============================] - 0s 207us/step - loss: 1.1921e-07 - accuracy: 0.1713
Epoch 9/39
1010/1010 [========================

In [68]:
y_train_predict = model.predict_classes(X_train) 
y_train = lb.inverse_transform(y_train)

accuracy_train = recall_score(y_train, y_train_predict, average='macro') 
print('-' * 35)
print('accuracy_train:', accuracy_train)


y_test_predict = model.predict_classes(X_test)
y_test = lb.inverse_transform(y_test)


accuracy_test = recall_score(y_test, y_test_predict, average='macro')

print('-' * 35)
print('accuracy_test:', accuracy_test)

-----------------------------------
accuracy_train: 0.9915771831453458
-----------------------------------
accuracy_test: 0.7559544407574811


In [47]:
print(X_test)
print(y_test_predict)

[[ 0.          0.          0.         ...  0.          2.83286071
   0.        ]
 [ 0.          0.          0.         ...  0.         38.81800461
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         33.71041107
   0.        ]]
[4 1 0 4 2 3 0 4 1 3 3 3 0 5 0 2 3 3 3 4 5 4 0 5 4 4 5 1 0 5 1 4 1 3 1 1 2
 3 3 4 4 1 5 5 3 0 5 1 0 3 5 5 4 4 1 3 4 3 4 3 3 0 5 4 0 3 0 3 5 3 3 4 3 0
 4 3 3 4 2 1 5 4 4 1 1 3 4 3 1 4 4 1 0 0 0 5 3 5 1 1 5 5 0 0 1 3 1 1 1 0 0
 1 3 1 4 3 4 5 1 1 0 0 2 1 4 4 4 5 4 0 4 3 4 5 4 3 5 3 2 0 1 5 4 3 5 5 3 1
 4 0 0 2 5 4 3 3 3 3 0 3 5 4 0 5 0 2 5 1 1 3 4 0 1 3 1 4 4 5 5 5 3 2 4 5 3
 0 0 1 4 1 4 4 0 2 2 3 5 4 4 1 1 1 0 0 4 1 0 4 0 1 5 1 1 1 4 5 4 4 1 3 4 1
 3 3 0 0 0 3 0 1 0 1 1 5 1 5 5 4 3 4 4 1 0 1 1 1 4 4 2 3 4 4 3]


In [49]:
test_path = 'C:/Users/sheny/Desktop/Module3/te1/te/'

transform_format('/'.join(test_path.split('/')[:-2]))

# folders = os.listdir(test_path)  
# num = len(folders)
label_index = {'cardboard': 1, 'glass': 2, 'mental': 3, 'paper': 4, 'plastic': 5,'landfill_trash': 6}


In [50]:
print(len(folders))
new_test_path ='C:/Users/sheny/Desktop/Module3/te1/'
z = read_data(new_test_path)
result = model.predict_classes(z)
# final_result =[]
len(result)

6


1264

In [51]:
from pandas import *
idx = Int64Index(range(1264,2528))

data_file = {'Id':idx,'Prediction':result
    
}
df = DataFrame(data_file,columns =['Id','Prediction'])


In [52]:
# df['Prediction'] =df['Prediction'].replace(0,"cardboard")
replace_value ={0:'cardboard',1:'glass',2:'landfill_trash'}
df = df.replace({"Prediction":replace_value})

# df.replace({'Prediction':{'cardboard':1}},{'Prediction':{'glass':2}},{'Prediction':{'landfill_trash':6}})

In [53]:
replace_value2 ={'cardboard':1,'glass':2,'landfill_trash':6}
df = df.replace({"Prediction":replace_value2})

In [54]:
df.to_csv('kaggle_vgg_2.csv',header=True,index=False)